In [1]:
!pip install tensorflow-hub
!pip install tensorflow-datasets
!git clone https://github.com/beringresearch/lrfinder/
!cd lrfinder && python3 -m pip install .

Cloning into 'lrfinder'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 54 (delta 12), reused 52 (delta 10), pack-reused 0
Unpacking objects: 100% (54/54), done.
Processing /content/lrfinder
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for lrfinder: filename=lrfinder-0.1-py3-none-any.whl size=2487 sha256=ebdc59290ff6bbc3c2cf3a6d914de4c955fe6f1fde576821c0fd0d7d5183be9c
  Stored in directory: /tmp/pip-ephem-wheel-cache-4ksrdjnh/wheels/86/9d/6e/77716e502c767656047e5d92c0e2daa34b83316533a1b339bc
Successfully built lrfinder


In [7]:
import os
import numpy as np

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from lrfinder import LRFinder
K = tf.keras.backend

In [4]:
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)
train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

for s,l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())
  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete5LQC7U/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete5LQC7U/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [5]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = ""

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok) 
tokenizer.fit_on_texts(training_sentences) 
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences) 
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

Using LR finder to get best learning rate and then setting it through backend.

In [9]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32)),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

BATCH = 64
train_ds = tf.data.Dataset.from_tensor_slices((padded, training_labels_final))
train_ds = train_ds.batch(BATCH)

STEPS_PER_EPOCH = np.ceil(len(train_data) / BATCH)
lr_finder = LRFinder(model)
lr_finder.find(train_ds, start_lr=1e-6, end_lr=1, epochs=10,
               steps_per_epoch=STEPS_PER_EPOCH)
               
learning_rates = lr_finder.get_learning_rates()
losses = lr_finder.get_losses()

best_lr = lr_finder.get_best_lr(sma=20)
K.set_value(model.optimizer.lr, best_lr)

Epoch 1/10
391/391 [==============================] - 42s 97ms/step - loss: 0.6931 - accuracy: 0.5079
Epoch 2/10
391/391 [==============================] - 38s 98ms/step - loss: 0.6930 - accuracy: 0.5121
Epoch 3/10
391/391 [==============================] - 39s 99ms/step - loss: 0.6928 - accuracy: 0.5223
Epoch 4/10
391/391 [==============================] - 38s 98ms/step - loss: 0.6874 - accuracy: 0.5556
Epoch 5/10
391/391 [==============================] - 39s 99ms/step - loss: 0.4119 - accuracy: 0.8187
Epoch 6/10
391/391 [==============================] - 39s 100ms/step - loss: 0.3304 - accuracy: 0.8626
Epoch 7/10
391/391 [==============================] - 38s 98ms/step - loss: 0.3171 - accuracy: 0.8681
Epoch 8/10
391/391 [==============================] - 39s 99ms/step - loss: 0.3881 - accuracy: 0.8010
Epoch 9/10
391/391 [==============================] - 39s 99ms/step - loss: 0.6966 - accuracy: 0.4983
Epoch 10/10
391/391 [==============================] - 39s 100ms/step - loss: 0.7

In [10]:
# Task of Day 4, Earlystopping with Patience 3

from tensorflow.keras.callbacks import EarlyStopping

earlystop_callback = EarlyStopping(
  monitor='val_loss', min_delta=0.0001, patience=3)

In [11]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final), callbacks=[earlystop_callback])
model.evaluate(testing_padded)

Epoch 1/10
782/782 [==============================] - 77s 99ms/step - loss: 0.6926 - accuracy: 0.5156 - val_loss: 0.6914 - val_accuracy: 0.5564
Epoch 2/10
782/782 [==============================] - 77s 98ms/step - loss: 0.6688 - accuracy: 0.5901 - val_loss: 0.5263 - val_accuracy: 0.7489
Epoch 3/10
782/782 [==============================] - 78s 99ms/step - loss: 0.4022 - accuracy: 0.8209 - val_loss: 0.3770 - val_accuracy: 0.8323
Epoch 4/10
782/782 [==============================] - 77s 99ms/step - loss: 0.2648 - accuracy: 0.8956 - val_loss: 0.4041 - val_accuracy: 0.8354
Epoch 5/10
782/782 [==============================] - 77s 99ms/step - loss: 0.1953 - accuracy: 0.9286 - val_loss: 0.4206 - val_accuracy: 0.8332
Epoch 6/10
782/782 [==============================] - 2s 1ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00


[0.0, 0.0]

Early Stopping stopped the training after 6 epochs of increase in lose thus saving from overfitting.